In [1]:
import docx

In [2]:
def get_tests(table):
    units = {}
    data = {}
    referent_col_name =''
    rows_num = len(table.rows)
    cells_num = len(table.rows[1].cells)
    blank_rows_ind = []
    is_referent_col = False  #referent column flag ("ед.измер", "Норма")
    
    #get referent name ("ед.измер", "Норма")
    if table.rows[1].cells[1].text == 'ед.изм' or table.rows[1].cells[1].text == 'Норма':
        referent_col_name = table.rows[1].cells[1].text
        is_referent_col = True

    #get blank rows
    for i in range(rows_num):
        row = table.rows[i]
        if (row.cells[0].text == '' and row.cells[1].text == ''):
            blank_rows_ind.append(i)
    blank_rows_ind.append(len(table.rows))

    #get sample units
    if is_referent_col:
        for row in range(1, rows_num):
            unit_name = table.rows[row].cells[0].text
            if unit_name != '':
                units[table.rows[row].cells[0].text] = table.rows[row].cells[1].text
            else: 
                continue
    
    #get samples results
    start_row = 0
    start_col = 2 if is_referent_col else 1
    for blank in blank_rows_ind:
        for cell in range(start_col, cells_num):
            date = table.rows[start_row+1].cells[cell].text.strip()
            if date == '':
                continue
            test = {}
            for row in range (start_row+2, blank):
                observation = table.rows[row].cells[cell].text.strip()
                if observation != '':
                    observation_name = table.rows[row].cells[0].text.strip()
                    test[observation_name] = observation
                else: 
                    continue
            data[date] = test
        start_row = blank
    return{'referent_col_name': referent_col_name, 'units': units, 'data': data}

In [3]:
doc = docx.Document('../data/123.docx')

In [4]:
for table in doc.tables:
    print (table.rows[0].cells[0].text)

ФИО
Общие анализы крови
Биохимические анализы крови
Общие анализы мочи
Иммунологические исследования
Дополнительные исследования
Микробиологические посевы
Этиотропная терапия
Температурный лист


In [52]:
tables_mapping = {'ФИО': 'general_info', 
                  'Общие анализы крови': 'cbc',
                  'Биохимические анализы крови': 'blood_tests',
                  'Общие анализы мочи': 'urine_tests',
                  'Иммунологические исследования': 'immunologic_tests',
                  'Дополнительные исследования': 'additional_tests',
                  'Микробиологические посевы': 'microbiology',
                  'Этиотропная терапия': 'antibiotics',
                  'Температурный лист': 'temperature'}

In [53]:
tables = {}
for table in doc.tables:
    tables[tables_mapping[table.rows[0].cells[0].text.strip()]] = table
tables

{'general_info': <docx.table.Table at 0x9b3a10>,
 'cbc': <docx.table.Table at 0x9b3db0>,
 'blood_tests': <docx.table.Table at 0x9b3890>,
 'urine_tests': <docx.table.Table at 0x9b32b0>,
 'immunologic_tests': <docx.table.Table at 0x9b3ad0>,
 'additional_tests': <docx.table.Table at 0x9b3ab0>,
 'microbiology': <docx.table.Table at 0x9b3310>,
 'antibiotics': <docx.table.Table at 0x9b35f0>,
 'temperature': <docx.table.Table at 0x9b3550>}

In [8]:
#get temperature data
temperature = {}
the_table = tables['temperature']
rows_num = len(the_table.rows)
cells_num = len(the_table.rows[1].cells)
temp_id = 0
for row in range(1, rows_num, 3):
    for cell in range(0, cells_num):
        date = the_table.rows[row].cells[cell].text
        if date != '':
            temperature[temp_id] = {'date':date,
                                    'temp': the_table.rows[row+1].cells[cell].text}
            temp_id+=1
        else: 
            break

In [15]:
#get general info
gen_info_mapping = {'ФИО': 'name',
                    'Дата рождения': 'birthdate',
                    'Адрес домашний': 'address',
                    'Отделение': 'department',
                    'Дата госпитализации': 'admission_date',
                    'Дата выписки': 'discharge_date',
                    'Клинический диагноз': 'diagnosis'}
general_info = {}
the_table = tables['general_info']
for row in the_table.rows:
    general_info[gen_info_mapping[row.cells[0].text.strip()]] = row.cells[1].text
general_info

{'name': 'ЙЦУ',
 'birthdate': '11.11.1988',
 'address': 'ФЫВА.',
 'department': 'РАО -> 2 хирургическое',
 'admission_date': '04.09.2018',
 'discharge_date': '05.12.2018',
 'diagnosis': 'ЖКБ. Хронический калькулезный холецистит. Холедохолитиаз. Механическая желтуха. Острый некротический билиарный панкреатит. Инфицированный панкреонекроз. Гнойный оментобурсит. Парапанкреальная флегмона слева. Некроз брыжейки поперечно-ободочной кишки. Острая язва желудка, осложненная кровотечением от 11.09.2018г. Сепсис. Сепстический шок от 07.09.2018г. Левосторонний реактивный. Полифакторная анемия средней степени. Первичный гипотиреоз, ремиссия.'}

In [49]:
#get Complete blood count
cbc = get_tests(tables['cbc'])

In [5]:
cbc

NameError: name 'cbc' is not defined

In [42]:
#get Blood tests
blood_tests = get_tests(tables['blood_tests'])

In [6]:
blood_tests

NameError: name 'blood_tests' is not defined

In [44]:
#get Urine tests
urine_tests = get_tests(tables['urine_tests'])
# urine_tests = get_tests(doc.tables[3])

In [7]:
urine_tests

NameError: name 'urine_tests' is not defined

In [54]:
#get Immunologic tests
immunologic_tests = get_tests(tables['immunologic_tests'])

In [8]:
immunologic_tests

NameError: name 'immunologic_tests' is not defined